In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

from tqdm.notebook import tqdm

%matplotlib inline

In [2]:
pairs = np.load("pairs.npy")

In [6]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.layers import *

In [7]:
class Normalize(keras.layers.Layer):
    """
    Normalization Layer For Images
    """
    def __init__(self,):
        super(Normalize,self).__init__()
        
    def __call__(self,x):
        return x/255

In [8]:
_input = Input(shape=(128,128,3))
norm = Normalize()(_input)

conv_0 = Conv2D(32,3,activation="relu",padding="same")(norm)
conv_1 = Conv2D(32,3,activation="relu",padding="same")(concatenate([norm,conv_0]))
conv_2 = Conv2D(32,1,activation="relu",padding="same")(concatenate([norm,conv_0,conv_1]))

norm_1 = BatchNormalization()(concatenate([norm,conv_0,conv_1,conv_2]))
pool_1 = MaxPool2D()(norm_1)

conv_3 = Conv2D(32,3,activation="relu",padding="same")(pool_1)
conv_4 = Conv2D(32,3,activation="relu",padding="same")(concatenate([pool_1,conv_3]))
conv_5 = Conv2D(32,1,activation="relu",padding="same")(concatenate([pool_1,conv_3,conv_4]))

norm_2 = BatchNormalization()(concatenate([pool_1,conv_3,conv_4,conv_5]))
pool_2 = MaxPool2D()(norm_2)

conv_6 = Conv2D(64,3,activation="relu",padding="same")(pool_2)
conv_7 = Conv2D(64,3,activation="relu",padding="same")(concatenate([pool_2,conv_6]))
conv_8 = Conv2D(64,1,activation="relu",padding="same")(concatenate([pool_2,conv_6,conv_7]))

norm_3 = BatchNormalization()(concatenate([pool_2,conv_6,conv_7,conv_8]))
pool_3 = MaxPool2D()(norm_3)

conv_9 = Conv2D(64,3,activation="relu",padding="same")(pool_3)
conv_10 = Conv2D(64,3,activation="relu",padding="same")(concatenate([pool_3,conv_9]))
conv_11 = Conv2D(64,1,activation="relu",padding="same")(concatenate([pool_3,conv_9,conv_10]))

norm_4 = BatchNormalization()(concatenate([pool_3,conv_9,conv_10,conv_11]))
pool_4 = MaxPool2D()(norm_4)

flat = Flatten()(pool_4)
dense_1 = Dense(2048)(flat)
dense_1 = LeakyReLU()(dense_1)
dense_2 = Dense(1024)(dense_1)
dense_2 = LeakyReLU()(dense_2)
out = Dense(128,activation="linear")(dense_2)

net = keras.Model(_input,out)

In [33]:
a_inp = Input(shape=(128,128,3),name="anc")
p_inp = Input(shape=(128,128,3),name="pos")
n_inp = Input(shape=(128,128,3),name="neg")

a_net = net(a_inp)
p_net = net(p_inp)
n_net = net(n_inp)

train = keras.Model([a_inp,p_inp,n_inp],concatenate([a_net,p_net,n_net]))

In [34]:
train.output_shape

(None, 384)

### Triplet 

![](triplet.png)

In [49]:
class Triplet(object):
    """
    Triplet Loss
    """
    def __init__(self,margin=0.5,dist="mse"):
        self.margin = margin
        self.dist = self.__getattribute__(dist)
    
    def mse(self,x,y):
        return tf.square(tf.subtract(x,y))
    
    def __call__(self,y_true,y_pred,*args,**kwargs):
        a_pred,p_pred,n_pred = tf.unstack(tf.reshape(y_true,(3,-1,128)))
        Dp = self.dist(a_pred,p_pred)
        Dn = self.dist(a_pred,n_pred)
        return Dp - Dn + 0.5

In [50]:
triplet = Triplet()

In [82]:
# y_pred = train((a[0:1].astype(np.float32),p[0:1].astype(np.float32),n[0:1].astype(np.float32)))

In [51]:
train.compile(optimizer="adam",loss=triplet)

NotImplementedError: Cannot convert a symbolic Tensor (model/Identity:0) to a numpy array.

In [54]:
train.fit((a,p,n),a_false.astype(np.float32),batch_size=1,epochs=1)

KeyboardInterrupt: 